In [166]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, String, Integer, Float, Date, Table, ForeignKey
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, relationship

import pandas as pd

In [167]:
credentials = pd.read_csv('../credentials/credentials.csv')

database_adress = credentials['Host'].iloc[0]
database_user = credentials['User'].iloc[0]
database_password = credentials['Password'].iloc[0]
database_name = credentials['Database'].iloc[0]
database_port = credentials['Port'].iloc[0]

print(database_user)

darth_vader


In [168]:
Base = automap_base()

# Create the middle Table movie_genres
movie_genres = Table(
    "movies_genres",
    Base.metadata,
    Column("movieId", ForeignKey("movies.movieId")),
    Column("genreId", ForeignKey("genres.genreId")),
)

#Create the class Genre with it's 2 properties genre_id and name
class Genre(Base):
   __tablename__ = 'genres' 
   genre_id = Column('genreId', Integer, primary_key = True)
   name = Column('genrename', String)

# Create the class Movie with it's 3 properites movie_id, title and release_year.
# Additionally connect the genres table with the movies table

class Genome_Tag(Base):
    __tablename__ = 'genometags' 
    tag_id = Column('tagId', Integer, primary_key = True)
    tag = Column('tag', String)


class Link(Base):
    __tablename__ = 'links' 
    movieId = Column(Integer, ForeignKey("movies.movieId"), primary_key = True)
    movie = relationship("Movie", back_populates="link", viewonly=True)
    imdb_id = Column('imdbId', Integer)
    tmdb_id = Column('tmdbId', Integer)

class Rating(Base):
    __tablename__ = 'ratings' 
    rating_id = Column('rating_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movieId = Column(Integer, ForeignKey("movies.movieId"))
    movie = relationship("Movie", back_populates="rating", viewonly=True)
    rating = Column('rating', Float)
    rating_date = Column('rating_date', Date)

class Tag(Base):
    __tablename__ = 'tags' 
    tag_id = Column('tag_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movieId = Column(Integer, ForeignKey("movies.movieId"))
    movie = relationship("Movie", back_populates="tag", viewonly=True)
    tag = Column('tag', String)
    tag_date = Column('tag_date', Date) 

class Genome_Score(Base):
    __tablename__ = 'genomescores' 
    tag_id = Column('tagId', Integer, primary_key = True)
    movieId = Column(Integer, ForeignKey("movies.movieId"))
    movie = relationship("Movie", back_populates="genome_score", viewonly=True)
    relevance = Column('relevance', Float)
    

class Movie(Base):
    __tablename__ = 'movies'

    movie_id = Column('movieid', Integer, primary_key = True)
    title = Column('title', String)
    release_year = Column('release_year', Integer)

    genres = relationship(Genre, secondary=movie_genres,
                            backref='genres', viewonly=True)
    ratings = relationship("Rating" ,back_populates='movie', viewonly=True)
    tags = relationship("Tag" ,back_populates='movie', viewonly=True)
    genome_scores = relationship("Genome_Score" ,back_populates='movie', viewonly=True)
    link = relationship("Link" ,back_populates='movie', viewonly=True, uselist=False)

In [169]:
#Build the connection with the database
engine = create_engine(f'mysql+pymysql://{database_user}:{database_password}@{database_adress}:{database_port}/{database_name}')
Base.prepare(autoload_with=engine)

session = Session(engine)

In [170]:
# Call the first Entry in the movies table and print out it's properties and all it's genres
movie_example = session.query(Movie).first()
print(f'Movie id: {movie_example.movie_id}')
print(f'Title: {movie_example.title}')
print(f'Release Year: {movie_example.release_year}')

print()
print("Genres: ")
for genre in list(movie_example.genres):
    print(genre.name)

print()
print(f'IMDb id: {movie_example.link.imdb_id}')
print(f'TMDb id: {movie_example.link.tmdb_id}')

i_ratings = 1
ratings = list(movie_example.ratings)
tags = list(movie_example.tags)
genome_scores = list(movie_example.genome_scores)
print()
print(f'First 10 ratings of movie {movie_example.title}')

while i_ratings <= 10:
    print(ratings[i_ratings].rating)
    if i_ratings >= 10:
        break
    i_ratings += 1

i_tags = 1
print()
print(f'First 10 tags of movie {movie_example.title}')

while i_tags <= 10:
    print(tags[i_tags].tag)
    if i_tags >= 10:
        break
    i_tags += 1

i_genome_scores = 1

print()
print(f'First 10 genome_scores of movie {movie_example.title}')

while i_genome_scores <= 10:
    print(genome_scores[i_genome_scores].relevance)
    if i_genome_scores >= 10:
        break
    i_genome_scores += 1

Movie id: 1
Title: Toy Story (1995)
Release Year: 1995

Genres: 
Adventure
Animation
Children
Comedy
Fantasy

IMDb id: 0114709
TMDb id: 862

First 10 ratings of movie Toy Story (1995)
5.0
4.5
4.0
4.0
4.0
5.0
5.0
4.5
5.0
4.0

First 10 tags of movie Toy Story (1995)
buddy movie
Cartoon
cgi
comedy
computer animation
family
friendship
kids
toy
toys

First 10 genome_scores of movie Toy Story (1995)
0.02375
0.05425
0.06875
0.16
0.19525
0.076
0.252
0.2275
0.024
0.587


In [171]:
# Call the first Entry in the genre table and print out it's properites
genome_tag_example = session.query(Genome_Tag).first()
print(f'Genome tag id: {genome_tag_example.tag_id}')
print(f'Genome tag: {genome_tag_example.tag}')

Genome tag id: 1
Genome tag: 007


In [172]:
link_example = session.query(Link).first()
print(f'Movie title: {link_example.movie.title}')
print(f'IMDb id: {link_example.imdb_id}')
print(f'TMDb id: {link_example.tmdb_id}')

Movie title: Toy Story (1995)
IMDb id: 0114709
TMDb id: 862


In [173]:
rating_example = session.query(Rating).first()
print(f'Rating id: {rating_example.rating_id}')
print(f'Rating user id: {rating_example.user_id}')
print(f'Rating movie title: {rating_example.movie.title}')
print(f'Rating: {rating_example.rating}')
print(f'Rating date: {rating_example.rating_date}')

Rating id: 1
Rating user id: 1
Rating movie title: Three Colors: Blue (Trois couleurs: Bleu) (1993)
Rating: 3.5
Rating date: 2009-10-27 09:00:21


In [174]:
tag_example = session.query(Tag).first()
print(f'Tag id: {tag_example.tag_id}')
print(f'Tag user id: {tag_example.user_id}')
print(f'Tag movie title: {tag_example.movie.title}')
print(f'Tag: {tag_example.tag}')
print(f'Tag date: {tag_example.tag_date}')



Tag id: 1
Tag user id: 14
Tag movie title: Braveheart (1995)
Tag: epic
Tag date: 2015-09-24 14:35:38


In [175]:
genome_score_example = session.query(Genome_Score).first()
print(f'Genome score tag id: {genome_score_example.tag_id}')
print(f'Genome score movie title: {genome_score_example.movie.title}')
print(f'Genome score relevance: {genome_score_example.relevance}')

Genome score tag id: 1
Genome score movie title: Toy Story (1995)
Genome score relevance: 0.029
